In [1]:
import os
import time
import pickle

import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [2]:
# 定数
op = Options()
# --headlessだけではOSによって動かない、プロキシが弾かれる、
# CUI用の省略されたHTMLが帰ってくるなどの障害が出ます。
# 長いですが、これら6行あって最強かつどんな環境でも動きますので、必ず抜かさないようにしてください。
op.add_argument("--disable-gpu");
op.add_argument("--disable-extensions");
op.add_argument("--proxy-server='direct://'");
op.add_argument("--proxy-bypass-list=*");
op.add_argument("--start-maximized");
op.add_argument("--headless");
driver = webdriver.Chrome(options=op)

In [3]:
# 関数
def rand_sleep():
    # 10秒〜30秒の間でランダムにスリープ
    time.sleep(np.random.randint(10, 22))

In [4]:
# driver.get(main_url)
# rand_sleep()
# soup = BeautifulSoup(driver.page_source, "html.parser")
# category_urls = []
# for ele in soup.find("div", attrs={"class", "navInner"}).find_all("li"):
#     category_urls.append(ele.find("a").get("href"))
# # https://news.livedoor.com/は外す
# category_urls.pop(0)
# category_urls

In [5]:
category_urls = [
    'https://news.livedoor.com/topics/category/main/',
 'https://news.livedoor.com/topics/category/dom/',
 'https://news.livedoor.com/topics/category/world/',
 'https://news.livedoor.com/topics/category/eco/',
 'https://news.livedoor.com/topics/category/ent/',
 'https://news.livedoor.com/topics/category/sports/',
 'https://news.livedoor.com/topics/category/gourmet/',
 'https://news.livedoor.com/topics/category/love/',
 'https://news.livedoor.com/topics/category/trend/',
]

In [7]:
# データを入れておき後でdataframeに変換
# すでにスクレイピング済みのデータがある場合はそっちを使う
try:
    with open('../data/article_infos.bin', 'rb') as f:
        article_infos = pickle.load(f)
except:
    article_infos = []
print(len(article_infos))
article_infos[-1]

8720


['https://news.livedoor.com/article/detail/20426010/',
 '2021年6月25日 11時32分',
 '「レジ台に座り、マスク無しで店員に暴言」 昼時のマクドナルドで見られた“子供たちの迷惑行為”が話題に',
 '\n「マクドナルドに行ったんですけど中学生と小学生の集団が店員さんに"え！？なに？？"、"は？？"ってタメ口で態度悪いし、飲み物受け取る時に"ベビタッピってやって！"って忙しすぎるお昼時なのに頼み、断られると"ノリわっる！"など複数人で店員さんに暴言を吐くという地獄の空間に遭遇しました」【写真】話題になった投稿はこちら…「地獄の空間に遭遇しました」昼時のマクドナルド某店で起こった子供たちの迷惑行為がSNS上で大きな注目を集めている。目撃者のねるねるねるねさん（@neruuneruuneru）によると子供たちは5人組。コロナ禍にもかかわらずマスクも付けず大声で店員に暴言を繰り返し、わざと広めにテーブルを占領。果てはレジ台の上に座るなど狼藉の限りを尽くしたということだ。\n\nこの件に限らず最近の子供たちのマナーが気にかかるという人は多いらしい。ねるねるねるねさんの投稿に対しSNSユーザー達からは「よく居ますよ、金持ってるから客で偉いんだと勘違いしてる子供、遊園地とかテーマパークじゃ長期休みの期間に増えますね、普段持たない金額を持つ事で気持ちが大きくなるのが必ず居ます( ・`д・´)」「コンビニで働いてた時平気でお札を投げる子供がいました\u3000別日に母親と来た時に母親がお金を投げたのを見て納得しました」「ちょっと前に小学生がふざけてバスの降車ボタンを毎回押しまくり、運行を妨害したとき、運転手が"押した方、降りて下さい"と降りるのを強要したと問題になりましたが、正直、それくらい厳しいほうがいいと思いました。ルールはルールときちんとした態度を取ったほうがいいと思います」「私、マックで働いてますが結構よくあります…( ᵕ̩̩ㅅᵕ̩̩ )今日も女子高生たちの集団が忙しい昼間に来て注文は普通でしたが席に着いた途端甲高い声で笑ったり話したりと店側としては迷惑でした…しかも忙しくて誰も注意しにいけず周りのお客さんも帰っていく人が多かったです」…など数々のコメントが寄せられている。\u3000\u3000\u3000

In [ ]:
# スクレピング開始
for category_url in category_urls[4:]:
    print("カテゴリ　{} からスクレイピング開始".format(category_url))
    # 初期変数
    pager_idx = 31
    # pager_idx = 126 # 前回125ページまで行ったため
    # sportは8ページまで
    # 既にスクレイピング済みのサイトのURL
    scraped_page_urls = set([article[0] for article in article_infos])
    
    # ex. https://news.livedoor.com/topics/category/main/?p=1 にアクセスして解析
    category_pager_url = "{0}?p={1}".format(category_url, pager_idx)
    driver.get(category_pager_url)
    print("{}のサイトから1つずつページをスクレイピング".format(category_pager_url))
    rand_sleep()
    # 大元のソープ
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # 次のページへがなくなるまで繰り返す
    while True:
    # 実際にデータを収集するページのリンクを抜き取る
        for ele in soup.find("div", attrs={"class": "mainBody"}).find_all("li"):
            # リンクを修正する
            page_url = ele.find("a").get("href").replace("topics", "article")
            # すでにスクレイピング済みだったら
            if page_url in scraped_page_urls:
                print("{}は既にスクレイピング済です".format(page_url))
                continue
            # ex. https://news.livedoor.com/article/detail/20558207/にアクセスする
            print("{}をスクレピング開始".format(page_url))
            driver.get(page_url)
            rand_sleep()
            
            # スクレピング対象の記事
            article_soup = BeautifulSoup(driver.page_source, "html.parser")
            # データ習得
            try:
                article_category = "<SEP>".join([c.text for c in article_soup.find("nav", attrs={"class": "breadcrumbs"}).find_all("span", attrs={"itemprop": "name"})])
            except:
                article_category = "None Category"
            try:
                article_keyword = "<SEP>".join([k.text for k in article_soup.find("ul", attrs={"class": "articleHeadKeyword"}).find_all("a")])
            except:
                article_keyword = "None Keyword"
            try:
                article_date = article_soup.find("time", attrs={"class": "articleDate"}).text
            except:
                article_date = "None Date"
            try:
                article_vender = article_soup.find("p", attrs={"class": "articleVender"}).find("span", attrs={"itemprop": "name"}).text.strip(" ").strip("\n")
            except:
                article_vender = "None Vender"
            try:
                article_tweet_counts = article_soup.find("ul", attrs={"class": "socialBtn"}).find("div", attrs={"class": "tweet_counts"}).text
            except:
                article_tweet_counts = 0
            try:
                article_title = article_soup.find("h1", attrs={"class": "articleTtl"}).text
            except:
                article_title = "None Title"
            try:
                article_text = article_soup.find("span", attrs={"itemprop": "articleBody"}).text
            except:
                article_text = "None Text"
            # データを一旦listに保存
            article_infos.append([page_url, article_date, article_title, article_text, article_category, article_keyword, article_tweet_counts, article_vender])
            print("{}の抽出完了".format(page_url))
            print("記事のタイトル: {0}, 記事の本文一部: {1}".format(article_title, article_text[0:10]))
        
        # ?p=〇ごとにデータ保存
        with open('../data/article_infos.bin', 'wb') as f:
            print("記事数{}をpickleに保存します".format(len(article_infos)))
            pickle.dump(article_infos , f)
            
        # 30ページスクレイピングしたら次のカテゴリへ
        if pager_idx > 50:
            print("次のカテゴリへ進みます")
            break
            
        # 次のページを進むのボタンがあれば
        if soup.find("ul", attrs={"class": "pager"}).find("li", attrs={"class": "next"}):
            pager_idx += 1
            category_pager_url = "{0}?p={1}".format(category_url, pager_idx)
            print("{0}ページ目の{1}に進みます".format(pager_idx, category_pager_url))
            driver.get(category_pager_url)
            rand_sleep()
            
            soup = BeautifulSoup(driver.page_source, "html.parser")
        # 最後のページのためループを抜ける
        else:
            print("次のカテゴリへ進みます")
            break
#     # 6ページ目まで行ったら一旦終了
#     if pager > 5:
#         break

カテゴリ　https://news.livedoor.com/topics/category/world/ からスクレイピング開始
https://news.livedoor.com/topics/category/world/?p=31のサイトから1つずつページをスクレイピング
https://news.livedoor.com/article/detail/20503913/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20503050/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20502928/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20503466/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20503018/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20502859/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20503052/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20502514/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20502595/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20501846/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20502108/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20502185/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20501437/は既にスクレイピング済です
https://news.livedoor.com/a

https://news.livedoor.com/article/detail/20482426/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20483634/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20478777/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20483726/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20483503/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20482631/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20482812/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20482292/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20483000/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20483001/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20482513/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20481939/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20480196/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20481519/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20481760/は既にスクレイピング済です
https://news.livedoor.com/article/detail

https://news.livedoor.com/article/detail/20465682/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20462251/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20464769/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20464364/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20462408/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20464749/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20464183/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20463335/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20460644/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20463767/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20463731/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20461840/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20463519/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20461843/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20463507/は既にスクレイピング済です
https://news.livedoor.com/article/detail

https://news.livedoor.com/article/detail/20444821/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20445002/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20444837/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20444181/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20444791/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20444793/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20444510/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20443268/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20443145/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20442265/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20443363/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20443198/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20442487/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20443003/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20441059/は既にスクレイピング済です
https://news.livedoor.com/article/detail

https://news.livedoor.com/article/detail/20465066/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20466028/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20465640/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20464760/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20463957/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20463339/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20463809/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20462465/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20459101/をスクレピング開始
https://news.livedoor.com/article/detail/20459101/の抽出完了
記事のタイトル: 年間50件以上も…！中国人が「京都の不動産」を爆買い中, 記事の本文一部: 
「中国人による日本
https://news.livedoor.com/article/detail/20463652/をスクレピング開始
https://news.livedoor.com/article/detail/20463652/の抽出完了
記事のタイトル: 25年ルール以外に「ゲーム」や「Youtube」の影響も！　国産旧車スポーツの中古価格「爆上げ」が止まらないワケ, 記事の本文一部: 
アメリカの「25年
https://news.livedoor.com/article/detail/20462992/をスクレピング開始
https://news.livedoor.com/article/detail/20462992/の

...

https://news.livedoor.com/article/detail/20336778/の抽出完了
記事のタイトル: クリスピー・クリーム・ドーナツのふわふわ“アザラシ”ドーナツ、スイカ風味の浮き輪ドーナツも, 記事の本文一部: 
クリスピー・クリー
https://news.livedoor.com/article/detail/20335081/をスクレピング開始
https://news.livedoor.com/article/detail/20335081/の抽出完了
記事のタイトル: SNSで話題の″バズるグルメ″ 第49回 台湾グルメが引き続き人気! 「#ルーロー飯」, 記事の本文一部: 
ネット社会の現代で
https://news.livedoor.com/article/detail/20333171/をスクレピング開始
https://news.livedoor.com/article/detail/20333171/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20337017/をスクレピング開始
https://news.livedoor.com/article/detail/20337017/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20336101/をスクレピング開始
https://news.livedoor.com/article/detail/20336101/の抽出完了
記事のタイトル: 【ペッパーランチ】テイクアウト限定『肉盛りプレート』！ ガーリックライスとの相性がヤバい, 記事の本文一部: 

ステーキ専門チェ
https://news.livedoor.com/article/detail/20336158/をスクレピング開始
https://news.livedoor.com/article/detail/20336158/の抽出完了
記事のタイトル: フレッシュでスパイシー！  手早くできてうまみたっぷり、生トマト×カレーの炒めもの5つ, 記事の本文

https://news.livedoor.com/article/detail/20325254/の抽出完了
記事のタイトル: 2,000円台のプチ贅沢！ ホテル最上階で味わえる期間限定「冷やし麺」, 記事の本文一部: 
ヨコハマ グランド
https://news.livedoor.com/article/detail/20325593/をスクレピング開始
https://news.livedoor.com/article/detail/20325593/の抽出完了
記事のタイトル: 【コストコ】驚きの声続出！スタバのタンブラーセットが1400円で買えるなんて！, 記事の本文一部: 

アメリカ生まれの
https://news.livedoor.com/article/detail/20325306/をスクレピング開始
https://news.livedoor.com/article/detail/20325306/の抽出完了
記事のタイトル: 仕事はかどりそう。森永ラムネ、ドリンクになる。, 記事の本文一部: 

森永製菓のお菓子
https://news.livedoor.com/article/detail/20325534/をスクレピング開始
https://news.livedoor.com/article/detail/20325534/の抽出完了
記事のタイトル: ほくほくポテトにとろ～りチーズ。レンチンで完成の、ご飯が進むメインおかず, 記事の本文一部: 
へとへとだけれど、
https://news.livedoor.com/article/detail/20324536/をスクレピング開始
https://news.livedoor.com/article/detail/20324536/の抽出完了
記事のタイトル: ローソンストア100、話題のスイーツ「マリトッツォ」が登場!, 記事の本文一部: 
ローソンストア10
https://news.livedoor.com/article/detail/20323344/をスクレピング開始
https://news.livedoor.com/article/detail/20323344/の抽出完了
記事のタイトル: 【ファミマ】低糖質でダイエット中もOK♡ 注目の「ヘルシーおやつ」まとめ,

https://news.livedoor.com/article/detail/20312372/の抽出完了
記事のタイトル: 切って焼くだけ！ワンパンでじゃがいものガレット, 記事の本文一部: 
      切って
記事数9661をpickleに保存します
51ページ目のhttps://news.livedoor.com/topics/category/gourmet/?p=51に進みます
https://news.livedoor.com/article/detail/20313222/をスクレピング開始
https://news.livedoor.com/article/detail/20313222/の抽出完了
記事のタイトル: バナナ好きに刺さる。ローソン「完熟バナナケーキ」の愛すべきポイントとは…, 記事の本文一部: 
      バナナ
https://news.livedoor.com/article/detail/20312814/をスクレピング開始
https://news.livedoor.com/article/detail/20312814/の抽出完了
記事のタイトル: おうち飲みに！今飲むべきクラフトビールを目利きのプロが厳選, 記事の本文一部: 
      おうち
https://news.livedoor.com/article/detail/20311108/をスクレピング開始
https://news.livedoor.com/article/detail/20311108/の抽出完了
記事のタイトル: ファミマ×湖池屋、「カラムーチョフェス」開催 - 7種類のコラボ商品を販売, 記事の本文一部: 
ファミリーマートは
https://news.livedoor.com/article/detail/20314343/をスクレピング開始
https://news.livedoor.com/article/detail/20314343/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20311659/をスクレピング開始
https://news.livedoor.com/article/detail/2

https://news.livedoor.com/article/detail/20431172/の抽出完了
記事のタイトル: 話題の「韓国コスメ」、実際どうなの？使用感をコスメライターが徹底レビュー！, 記事の本文一部: 

まだまだ熱が冷め
https://news.livedoor.com/article/detail/20427677/をスクレピング開始
https://news.livedoor.com/article/detail/20427677/の抽出完了
記事のタイトル: 【GU】1690円でゲットできるなんて！ ほっそり見えが叶う“ラッフルブラウス”はオンオフ問わず着回せそう, 記事の本文一部: 

最近は、気温が2
https://news.livedoor.com/article/detail/20430958/をスクレピング開始
https://news.livedoor.com/article/detail/20430958/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20429753/をスクレピング開始
https://news.livedoor.com/article/detail/20429753/の抽出完了
記事のタイトル: 無印良品の新商品「冷風扇」は、コンパクトなのにとっても涼しい！ エアコンが苦手な方におすすめです｜マイ定番スタイル, 記事の本文一部: 

        
https://news.livedoor.com/article/detail/20431121/をスクレピング開始
https://news.livedoor.com/article/detail/20431121/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20429796/をスクレピング開始
https://news.livedoor.com/article/detail/20429796/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
http

https://news.livedoor.com/article/detail/20417353/の抽出完了
記事のタイトル: 本命確定！これを見たら喜んでいいLINEの送信時間, 記事の本文一部: 

   好きな人と
https://news.livedoor.com/article/detail/20417415/をスクレピング開始
https://news.livedoor.com/article/detail/20417415/の抽出完了
記事のタイトル: 飽きた！以外のセックスレスの理由は結構ある。男子にレスの原因をリサーチしてみた, 記事の本文一部: 

独自の調べによる
記事数9736をpickleに保存します
36ページ目のhttps://news.livedoor.com/topics/category/love/?p=36に進みます
https://news.livedoor.com/article/detail/20417484/をスクレピング開始
https://news.livedoor.com/article/detail/20417484/の抽出完了
記事のタイトル: 「元彼が恋したのはシングルマザー」を受け入れられない女, 記事の本文一部: 

毎日がキラキラし
https://news.livedoor.com/article/detail/20416947/をスクレピング開始
https://news.livedoor.com/article/detail/20416947/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20416110/をスクレピング開始
https://news.livedoor.com/article/detail/20416110/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20415435/をスクレピング開始
https://news.livedoor.com/article/detail/20415435/の抽出完了
記事のタイトル: 無印良品「